### Create Binned Catalog V1

The work in this notebook is desined to bin/create a catalog for the ml Project, from the catalog generated by Thomas Li 3/9/2025
@marcoc7

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy
from astropy.table import Table
from astropy.io import fits

%load_ext autoreload

import sys
from pathlib import Path



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%autoreload 2

Load and Process Table:

In [3]:
sourceFile = '/mnt/md127/SPHEREx/spherex_nep_catalogues/photoz_and_spherex_flux.fits' #Change this as needed, this file is only for Rabbit
sourceCatalog = Table.read(sourceFile) 

In [ ]:
sourceCatalog

In [ ]:
print(sourceCatalog.colnames)


In [4]:
originalCat = Table.read('/mnt/md127/SPHEREx/spherex_nep_catalogues/processed_master_v5.fits') 
originalCat

Index,ra,dec,ls_bit,unwise_bit,h20_bit,sv3_bit,heroes_bit,spec_z,spec_zerr,ra_ls,ra_unwise,ra_h20,ra_sv3,ra_heroes,dec_ls,dec_unwise,dec_h20,dec_sv3,dec_heroes,ls_id,unwise_id,h20_id,sv3_id,heroes_id,mag_g_ls,mag_err_g_ls,mag_r_ls,mag_err_r_ls,mag_z_ls,mag_err_z_ls,mag_w1_unwise,mag_err_w1_unwise,mag_w2_unwise,mag_err_w2_unwise,mag_g_h20,mag_err_g_h20,mag_r_h20,mag_err_r_h20,mag_i_h20,mag_err_i_h20,mag_z_h20,mag_err_z_h20,mag_y_h20,mag_err_y_h20,mag_n816_h20,mag_err_n816_h20,mag_n921_h20,mag_err_n921_h20,mag_g_heroes,mag_err_g_heroes,mag_r_heroes,mag_err_r_heroes,mag_i_heroes,mag_err_i_heroes,mag_z_heroes,mag_err_z_heroes,mag_y_heroes,mag_err_y_heroes,mag_n816_heroes,mag_err_n816_heroes,mag_n921_heroes,mag_err_n921_heroes,ra_sls,dec_sls,sls_bit,sls_id,mag_ch1_sls,mag_err_ch1_sls,mag_ch2_sls,mag_err_ch2_sls
int64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes64,bytes64,bytes64,bytes64,bytes64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float32,float32,float32,float32
1,280.1747210123532,61.99439627328558,1,0,0,0,0,--,--,280.1747210123532,--,--,--,--,61.99439627328558,--,--,--,--,9907740126282882.0,nan,nan,nan,nan,23.99044919678641,0.20145859829277582,23.800842224350532,0.30720995269412,22.679171705356115,0.15258160002227,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
2,280.1796342378335,61.98339010133163,1,1,0,0,0,--,--,280.1797722840697,280.17949619159737,--,--,--,61.98339310897537,61.98338709368789,--,--,--,9907740126282928.0,2810p620o0018688,nan,nan,nan,22.87977210330378,0.06837255630420122,21.559946686857124,0.03804681033474791,19.84753593605873,0.011477639826100205,20.11660276971751,0.041313855672979155,20.37775003295475,0.10912526856698361,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
3,280.179166070106,61.98342684566471,1,1,0,0,0,--,--,280.17883594861473,280.17949619159737,--,--,--,61.98346659764154,61.98338709368789,--,--,--,9907740126282920.0,2810p620o0018688,nan,nan,nan,22.149762697059607,0.04283440143795978,21.55837925106109,0.046804218316510084,21.228348852594383,0.053713533679827244,20.11660276971751,0.041313855672979155,20.37775003295475,0.10912526856698361,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
4,280.19780824892035,61.985725218209566,1,0,0,0,0,--,--,280.19780824892035,--,--,--,--,61.985725218209566,--,--,--,--,9907740126283078.0,nan,nan,nan,nan,24.41724308471316,0.3713734344666367,23.295520251882003,0.20079909767137988,22.619762068476117,0.1710810325256104,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
5,280.19067293013745,61.98443205547751,1,1,0,0,0,--,--,280.190745559526,280.1906003007488,--,--,--,61.98441774815328,61.98444636280175,--,--,--,9907740126283024.0,2810p620o0018707,nan,nan,nan,24.417497025424815,0.3526536437087211,22.134911817801942,0.0808513008004384,20.36814800754165,0.022681352567631288,19.104227609206035,0.01845180694744452,19.851668529411057,0.06900563936752026,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
6,280.2031928552568,61.98854289098511,1,1,0,0,0,--,--,280.20307884102675,280.2033068694869,--,--,--,61.98855620825967,61.988529573710544,--,--,--,9907740126283124.0,2810p620o0018807,nan,nan,nan,23.137586442283986,0.12559226793275566,22.3026967114693,0.0891464023380282,21.160050716896418,0.05032275382701149,20.344977093147413,0.04951684954522131,20.226669189077278,0.0951399977459063

In [ ]:
for col in sourceCatalog.columns:
    if hasattr(sourceCatalog[col], 'mask') and np.any(sourceCatalog[col].mask):
        print(f"⚠️ Column {col} contains masked values!")


In [5]:
from astropy.table import Table  # Ensure this is imported

# Define the binning strategy
bins = [
    (1, 10),   # 10 channels
    (11, 20),  # 10 channels
    (21, 30),  # 10 channels
    (31, 40),  # 10 channels
    (41, 50),  # 10 channels
    (51, 60),  # 10 channels
    (61, 70),  # 10 channels
    (71, 80),  # 10 channels
    (81, 91),  # 11 channels
    (92, 102)  # 11 channels
]
# Creating the new table with binned bands and debugging each step
binned_data = {}

for i, (start, end) in enumerate(bins, start=1):
    selected_columns = [f"ch{j}" for j in range(start, end + 1)]
    #print (selected_columns)
    # Extract values and check for issues
    values = np.array([sourceCatalog[col] for col in selected_columns])

    #print(f"\nBand {i} - Columns: {selected_columns}")
    #print(f"Values shape: {values.shape}")
    
    # Compute mean
    band_mean = np.nanmean(values, axis=0)  # Ignores NaN values

    # Check for NaNs before storing
    if np.any(np.isnan(band_mean)):
        print(f"⚠️ Band {i} contains NaN values!")
    
    binned_data[f"band{i}"] = band_mean

# Convert to astropy.Table
binned_table = Table(binned_data)

# Final NaN check in binned_table
print("\nFinal check for NaN values in binned_table...")
for band in binned_table.colnames:
    if np.any(np.isnan(binned_table[band])):
        print(f"❌ Band {band} still contains NaN values!")
    else:
        print(f"✅ Band {band} is clean.")



Final check for NaN values in binned_table...
✅ Band band1 is clean.
✅ Band band2 is clean.
✅ Band band3 is clean.
✅ Band band4 is clean.
✅ Band band5 is clean.
✅ Band band6 is clean.
✅ Band band7 is clean.
✅ Band band8 is clean.
✅ Band band9 is clean.
✅ Band band10 is clean.


In [8]:
binned_table

band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,id,ra,dec,z_best,z_med,z_err_std,snr_per_filter
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,uint64,float64,float64,float64,float64,float64,float64
2.681520900686674e-05,3.048576225348575e-05,4.262261112106731e-05,5.407932189180571e-05,5.9616764561542935e-05,5.070858833667531e-05,4.086165978041212e-05,3.7154243476732574e-05,3.5661351391208783e-05,3.4767379294930216e-05,2,280.17962646484375,61.98339080810547,0.30530010018849013,0.3070000112056732,0.02167546741850781,8.988694190979004
5.411545241695075e-06,8.762154299498739e-06,1.5490201241545152e-05,1.7676723609592366e-05,1.8253706283506807e-05,2.3256594545698388e-05,2.755236119193311e-05,2.9913880375036387e-05,3.313240689251029e-05,3.464740964024784e-05,3,280.1791687011719,61.98342514038086,1.7943031981142665,1.7929999828338623,0.7636348248884954,5.643349647521973
1.9800170292624555e-05,3.565085085493113e-05,5.098289028396336e-05,6.790073840029793e-05,7.756522588685293e-05,9.12520180844763e-05,7.842414361315851e-05,5.336935850298715e-05,4.588528572930254e-05,4.4097709944449394e-05,5,280.190673828125,61.984432220458984,0.679355981752951,0.6740000247955322,0.07312436718989827,7.261623382568359
7.510220454562847e-06,1.150310060009293e-05,2.1039755066609233e-05,4.460379914448898e-05,5.878832381575617e-05,7.460214649641206e-05,9.412971835377377e-05,9.767767350602094e-05,0.00010009535898087219,0.00010548488206710966,9,280.18048095703125,61.98760986328125,2.6019985018422793,2.5139999389648438,0.7484448868558956,5.81927490234375
8.562705235041022e-05,8.011751240164219e-05,0.00010005936984396459,0.00011113417898552091,0.00012175907358478118,0.00011098486821829784,9.017115178636255e-05,8.274301618677838e-05,7.977027400907374e-05,7.802582936474643e-05,10,280.1800842285156,61.987831115722656,0.4170768654944781,0.4189999997615814,0.0067790824098721844,23.856380462646484
2.2033035957456458e-05,3.337165419660239e-05,4.920775023874026e-05,6.243579787890124e-05,7.289114820847241e-05,6.195681891131399e-05,4.653173590978113e-05,4.596466602770599e-05,3.836609154896328e-05,3.441837330986877e-05,11,280.15484619140625,61.96268081665039,0.2699319741423631,0.2630000114440918,0.17263437585354416,6.052838325500488
3.4913219181948615e-05,6.164957706929101e-05,9.045183728050391e-05,0.00011400111717445002,0.00014613313161225392,0.00011834347881993089,7.564400002685087e-05,7.144664219501903e-05,6.532508472259995e-05,5.729920453428324e-05,12,280.14947509765625,61.964317321777344,0.24544961424590403,0.23800000548362732,0.07912544380225438,9.083745002746582
9.264645814469688e-05,9.189179821675676e-05,8.875463409124713e-05,8.813442056562035e-05,9.546471824034616e-05,0.00012097472101063973,8.132825160616305e-05,0.00020132196527433206,8.577983740573803e-05,8.384796833406042e-05,21,280.16021728515625,61.98466873168945,4.990999917392784,0.0,5.923168878389097e-05,16.59946632385254
1.823168666343588e-05,3.93529311476616e-05,7.162970341369655e-05,0.00010566256342355037,0.00012450942854311196,9.467467823629672e-05,7.723939007801199e-05,7.590462878020318e-05,7.036948185677261e-05,6.387619405562913e-05,29,280.1792907714844,61.98844909667969,0.1218945147299026,0.09799999743700027,0.1077304220717167,9.050853729248047


In [6]:
# List of columns to append
columns_to_append = ["id", "ra", "dec", "z_best", "z_med", "z_err_std", "snr_per_filter"]

# Extract these columns from sourceCatalog and add them to binned_table
for col in columns_to_append:
    binned_table[col] = sourceCatalog[col]

print("\n✅ Specified columns have been successfully appended to binned_table.")



✅ Specified columns have been successfully appended to binned_table.


In [7]:
binned_table = binned_table[binned_table["snr_per_filter"] >= 5]


In [9]:
originalCat.rename_column("Index", "id")
originalCat

id,ra,dec,ls_bit,unwise_bit,h20_bit,sv3_bit,heroes_bit,spec_z,spec_zerr,ra_ls,ra_unwise,ra_h20,ra_sv3,ra_heroes,dec_ls,dec_unwise,dec_h20,dec_sv3,dec_heroes,ls_id,unwise_id,h20_id,sv3_id,heroes_id,mag_g_ls,mag_err_g_ls,mag_r_ls,mag_err_r_ls,mag_z_ls,mag_err_z_ls,mag_w1_unwise,mag_err_w1_unwise,mag_w2_unwise,mag_err_w2_unwise,mag_g_h20,mag_err_g_h20,mag_r_h20,mag_err_r_h20,mag_i_h20,mag_err_i_h20,mag_z_h20,mag_err_z_h20,mag_y_h20,mag_err_y_h20,mag_n816_h20,mag_err_n816_h20,mag_n921_h20,mag_err_n921_h20,mag_g_heroes,mag_err_g_heroes,mag_r_heroes,mag_err_r_heroes,mag_i_heroes,mag_err_i_heroes,mag_z_heroes,mag_err_z_heroes,mag_y_heroes,mag_err_y_heroes,mag_n816_heroes,mag_err_n816_heroes,mag_n921_heroes,mag_err_n921_heroes,ra_sls,dec_sls,sls_bit,sls_id,mag_ch1_sls,mag_err_ch1_sls,mag_ch2_sls,mag_err_ch2_sls
int64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes64,bytes64,bytes64,bytes64,bytes64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float32,float32,float32,float32
1,280.1747210123532,61.99439627328558,1,0,0,0,0,--,--,280.1747210123532,--,--,--,--,61.99439627328558,--,--,--,--,9907740126282882.0,nan,nan,nan,nan,23.99044919678641,0.20145859829277582,23.800842224350532,0.30720995269412,22.679171705356115,0.15258160002227,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
2,280.1796342378335,61.98339010133163,1,1,0,0,0,--,--,280.1797722840697,280.17949619159737,--,--,--,61.98339310897537,61.98338709368789,--,--,--,9907740126282928.0,2810p620o0018688,nan,nan,nan,22.87977210330378,0.06837255630420122,21.559946686857124,0.03804681033474791,19.84753593605873,0.011477639826100205,20.11660276971751,0.041313855672979155,20.37775003295475,0.10912526856698361,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
3,280.179166070106,61.98342684566471,1,1,0,0,0,--,--,280.17883594861473,280.17949619159737,--,--,--,61.98346659764154,61.98338709368789,--,--,--,9907740126282920.0,2810p620o0018688,nan,nan,nan,22.149762697059607,0.04283440143795978,21.55837925106109,0.046804218316510084,21.228348852594383,0.053713533679827244,20.11660276971751,0.041313855672979155,20.37775003295475,0.10912526856698361,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
4,280.19780824892035,61.985725218209566,1,0,0,0,0,--,--,280.19780824892035,--,--,--,--,61.985725218209566,--,--,--,--,9907740126283078.0,nan,nan,nan,nan,24.41724308471316,0.3713734344666367,23.295520251882003,0.20079909767137988,22.619762068476117,0.1710810325256104,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
5,280.19067293013745,61.98443205547751,1,1,0,0,0,--,--,280.190745559526,280.1906003007488,--,--,--,61.98441774815328,61.98444636280175,--,--,--,9907740126283024.0,2810p620o0018707,nan,nan,nan,24.417497025424815,0.3526536437087211,22.134911817801942,0.0808513008004384,20.36814800754165,0.022681352567631288,19.104227609206035,0.01845180694744452,19.851668529411057,0.06900563936752026,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,--,--,--,--,--
6,280.2031928552568,61.98854289098511,1,1,0,0,0,--,--,280.20307884102675,280.2033068694869,--,--,--,61.98855620825967,61.988529573710544,--,--,--,9907740126283124.0,2810p620o0018807,nan,nan,nan,23.137586442283986,0.12559226793275566,22.3026967114693,0.0891464023380282,21.160050716896418,0.05032275382701149,20.344977093147413,0.04951684954522131,20.226669189077278,0.09513999774590638,-

In [10]:
from astropy.table import join

columns_to_append = [
    "ls_bit", "unwise_bit", "h20_bit", "sv3_bit", "heroes_bit",
    "ls_id", "sls_bit", "unwise_id", "h20_id", "sv3_id", "heroes_id", "sls_id"
]

# Perform the merge based on matching id in binned_table and Index in originalCat
mergedTablev1 = join(binned_table, originalCat["id", *columns_to_append], keys="id", join_type="left")


In [11]:
mergedTablev1

band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,id,ra,dec,z_best,z_med,z_err_std,snr_per_filter,ls_bit,unwise_bit,h20_bit,sv3_bit,heroes_bit,ls_id,sls_bit,unwise_id,h20_id,sv3_id,heroes_id,sls_id
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,bytes64,int64,bytes64,bytes64,bytes64,bytes64,float64
2.681520900686674e-05,3.048576225348575e-05,4.262261112106731e-05,5.407932189180571e-05,5.9616764561542935e-05,5.070858833667531e-05,4.086165978041212e-05,3.7154243476732574e-05,3.5661351391208783e-05,3.4767379294930216e-05,2.0,280.17962646484375,61.98339080810547,0.30530010018849013,0.3070000112056732,0.02167546741850781,8.988694190979004,1,1,0,0,0,9907740126282928.0,0,2810p620o0018688,nan,nan,nan,--
5.411545241695075e-06,8.762154299498739e-06,1.5490201241545152e-05,1.7676723609592366e-05,1.8253706283506807e-05,2.3256594545698388e-05,2.755236119193311e-05,2.9913880375036387e-05,3.313240689251029e-05,3.464740964024784e-05,3.0,280.1791687011719,61.98342514038086,1.7943031981142665,1.7929999828338623,0.7636348248884954,5.643349647521973,1,1,0,0,0,9907740126282920.0,0,2810p620o0018688,nan,nan,nan,--
1.9800170292624555e-05,3.565085085493113e-05,5.098289028396336e-05,6.790073840029793e-05,7.756522588685293e-05,9.12520180844763e-05,7.842414361315851e-05,5.336935850298715e-05,4.588528572930254e-05,4.4097709944449394e-05,5.0,280.190673828125,61.984432220458984,0.679355981752951,0.6740000247955322,0.07312436718989827,7.261623382568359,1,1,0,0,0,9907740126283024.0,0,2810p620o0018707,nan,nan,nan,--
7.510220454562847e-06,1.150310060009293e-05,2.1039755066609233e-05,4.460379914448898e-05,5.878832381575617e-05,7.460214649641206e-05,9.412971835377377e-05,9.767767350602094e-05,0.00010009535898087219,0.00010548488206710966,9.0,280.18048095703125,61.98760986328125,2.6019985018422793,2.5139999389648438,0.7484448868558956,5.81927490234375,1,1,0,0,0,9907740126282936.0,0,2810p620o0056304,nan,nan,nan,--
8.562705235041022e-05,8.011751240164219e-05,0.00010005936984396459,0.00011113417898552091,0.00012175907358478118,0.00011098486821829784,9.017115178636255e-05,8.274301618677838e-05,7.977027400907374e-05,7.802582936474643e-05,10.0,280.1800842285156,61.987831115722656,0.4170768654944781,0.4189999997615814,0.0067790824098721844,23.856380462646484,1,1,0,0,0,9907740126282932.0,0,2810p620o0056304,nan,nan,nan,--
2.2033035957456458e-05,3.337165419660239e-05,4.920775023874026e-05,6.243579787890124e-05,7.289114820847241e-05,6.195681891131399e-05,4.653173590978113e-05,4.596466602770599e-05,3.836609154896328e-05,3.441837330986877e-05,11.0,280.15484619140625,61.96268081665039,0.2699319741423631,0.2630000114440918,0.17263437585354416,6.052838325500488,1,1,0,0,0,9907740126282702.0,0,2810p620o0056064,nan,nan,nan,--
3.4913219181948615e-05,6.164957706929101e-05,9.045183728050391e-05,0.00011400111717445002,0.00014613313161225392,0.00011834347881993089,7.564400002685087e-05,7.144664219501903e-05,6.532508472259995e-05,5.729920453428324e-05,12.0,280.14947509765625,61.964317321777344,0.24544961424590403,0.23800000548362732,0.07912544380225438,9.083745002746582,1,1,0,0,0,9907740126282640.0,0,2810p620o0070791,nan,nan,nan,--
9.264645814469688e-05,9.189179821675676e-05,8.875463409124713e-05,8.813442056562035e-05,9.546471824034616e-05,0.00012097472101063973,8.132825160616305e-05,0.00020132196527433206,8.577983740573803e-05,8.384796833406042e-05,21.0,280.16021728515625,61.98466873168945,4.990999917392784,0.0,5.923168878389097e-05,16.59946632385254,1,1,0,0,0,9907740126282748.0,0,2810p620o0018708,nan,nan,nan,--
1.823168666343588e-05,3.93529311476616e-05,7.162970341369655e-05,0.00010566256342355037,0.00012450942854311196,9.467467823629672e-05,7.723939007801199e-05,7.590462878020318e-05,7.036948185677261e-05,6.387619405562913e-05,29.0,280.1792907714844,61.98844909667969,0.1218945147299026,0.09799999743700027,0.1077304220717167,9.050853729248047,1,1,0,0,0,99077401

In [12]:
# Ensure 'id' exists in mergedTablev1 before converting
if "id" in mergedTablev1.colnames:
    mergedTablev1["id"] = mergedTablev1["id"].astype(np.int64)
    print("\n✅ Column 'id' has been converted from float64 to int64.")
else:
    print("\n⚠️ Column 'id' not found in mergedTablev1.")



✅ Column 'id' has been converted from float64 to int64.


In [18]:
mergedTablev1

band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,id,ra,dec,z_best,z_med,z_err_std,snr_per_filter,ls_bit,unwise_bit,h20_bit,sv3_bit,heroes_bit,ls_id,sls_bit,unwise_id,h20_id,sv3_id,heroes_id,sls_id
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,bytes64,int64,bytes64,bytes64,bytes64,bytes64,float64
2.681520900686674e-05,3.048576225348575e-05,4.262261112106731e-05,5.407932189180571e-05,5.9616764561542935e-05,5.070858833667531e-05,4.086165978041212e-05,3.7154243476732574e-05,3.5661351391208783e-05,3.4767379294930216e-05,2,280.17962646484375,61.98339080810547,0.30530010018849013,0.3070000112056732,0.02167546741850781,8.988694190979004,1,1,0,0,0,9907740126282928.0,0,2810p620o0018688,nan,nan,nan,--
5.411545241695075e-06,8.762154299498739e-06,1.5490201241545152e-05,1.7676723609592366e-05,1.8253706283506807e-05,2.3256594545698388e-05,2.755236119193311e-05,2.9913880375036387e-05,3.313240689251029e-05,3.464740964024784e-05,3,280.1791687011719,61.98342514038086,1.7943031981142665,1.7929999828338623,0.7636348248884954,5.643349647521973,1,1,0,0,0,9907740126282920.0,0,2810p620o0018688,nan,nan,nan,--
1.9800170292624555e-05,3.565085085493113e-05,5.098289028396336e-05,6.790073840029793e-05,7.756522588685293e-05,9.12520180844763e-05,7.842414361315851e-05,5.336935850298715e-05,4.588528572930254e-05,4.4097709944449394e-05,5,280.190673828125,61.984432220458984,0.679355981752951,0.6740000247955322,0.07312436718989827,7.261623382568359,1,1,0,0,0,9907740126283024.0,0,2810p620o0018707,nan,nan,nan,--
7.510220454562847e-06,1.150310060009293e-05,2.1039755066609233e-05,4.460379914448898e-05,5.878832381575617e-05,7.460214649641206e-05,9.412971835377377e-05,9.767767350602094e-05,0.00010009535898087219,0.00010548488206710966,9,280.18048095703125,61.98760986328125,2.6019985018422793,2.5139999389648438,0.7484448868558956,5.81927490234375,1,1,0,0,0,9907740126282936.0,0,2810p620o0056304,nan,nan,nan,--
8.562705235041022e-05,8.011751240164219e-05,0.00010005936984396459,0.00011113417898552091,0.00012175907358478118,0.00011098486821829784,9.017115178636255e-05,8.274301618677838e-05,7.977027400907374e-05,7.802582936474643e-05,10,280.1800842285156,61.987831115722656,0.4170768654944781,0.4189999997615814,0.0067790824098721844,23.856380462646484,1,1,0,0,0,9907740126282932.0,0,2810p620o0056304,nan,nan,nan,--
2.2033035957456458e-05,3.337165419660239e-05,4.920775023874026e-05,6.243579787890124e-05,7.289114820847241e-05,6.195681891131399e-05,4.653173590978113e-05,4.596466602770599e-05,3.836609154896328e-05,3.441837330986877e-05,11,280.15484619140625,61.96268081665039,0.2699319741423631,0.2630000114440918,0.17263437585354416,6.052838325500488,1,1,0,0,0,9907740126282702.0,0,2810p620o0056064,nan,nan,nan,--
3.4913219181948615e-05,6.164957706929101e-05,9.045183728050391e-05,0.00011400111717445002,0.00014613313161225392,0.00011834347881993089,7.564400002685087e-05,7.144664219501903e-05,6.532508472259995e-05,5.729920453428324e-05,12,280.14947509765625,61.964317321777344,0.24544961424590403,0.23800000548362732,0.07912544380225438,9.083745002746582,1,1,0,0,0,9907740126282640.0,0,2810p620o0070791,nan,nan,nan,--
9.264645814469688e-05,9.189179821675676e-05,8.875463409124713e-05,8.813442056562035e-05,9.546471824034616e-05,0.00012097472101063973,8.132825160616305e-05,0.00020132196527433206,8.577983740573803e-05,8.384796833406042e-05,21,280.16021728515625,61.98466873168945,4.990999917392784,0.0,5.923168878389097e-05,16.59946632385254,1,1,0,0,0,9907740126282748.0,0,2810p620o0018708,nan,nan,nan,--
1.823168666343588e-05,3.93529311476616e-05,7.162970341369655e-05,0.00010566256342355037,0.00012450942854311196,9.467467823629672e-05,7.723939007801199e-05,7.590462878020318e-05,7.036948185677261e-05,6.387619405562913e-05,29,280.1792907714844,61.98844909667969,0.1218945147299026,0.09799999743700027,0.1077304220717167,9.050853729248047,1,1,0,0,0,9907740126282922.0,0,2810p62

In [21]:
SubaruTable = Table.read('/home/marco/lsHEROESmergedv2')
SubaruTable

ra_heroes,dec_heroes,ra_ls,dec_ls,from,ls_id,hsc_id,type,i_ap2,i_ap2err,i_kron,i_kronerr,z_ap2,z_ap2err,z_kron,z_kronerr,flux_z,mag_z,snr_z,y_ap2,y_ap2err,y_kron,y_kronerr,n816_ap2,n816_ap2err,n816_kron,n816_kronerr,n921_ap2,n921_ap2err,n921_kron,n921_kronerr,mag_w1,flux_w1,snr_w1,mag_w2,flux_w2,snr_w2,mag_w3,flux_w3,snr_w3,mag_w4,flux_w4,snr_w4,x,y,xx,yy,xy,patch,wise_coadd_id,brickid,brickname,sersic,shape_r,shape_e1,shape_e2,ellipticity,theta,sc_id
float64,float64,float64,float64,bytes6,int64,int64,bytes3,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,bytes5,bytes8,int64,bytes8,float64,float64,float64,float64,float64,float64,int64
278.53500138043825,63.9749665047913,--,--,HEROES,--,158913789953,--,--,--,--,--,23.559645,0.21461079,--,--,--,--,--,-23.850363,0.7138585,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19949.146,59448.258,--,--,--,"1,5",--,--,--,--,--,--,--,--,--,--
278.54114302999335,63.97458600247457,--,--,HEROES,--,158913789954,--,--,--,--,--,24.353443,0.44712862,23.963285,0.24130464,--,--,--,23.672665,0.61583346,25.028233,1.6664253,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19890.473,59448.87,3.136412,3.7544372,1.5408987,"1,5",--,--,--,--,--,--,--,--,--,--
278.53467936808585,63.98020507322936,--,--,HEROES,--,158913789960,--,--,--,--,--,23.596027,0.21847671,23.608713,0.2265859,--,--,--,23.064314,0.2341843,23.138815,0.26028684,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19969.445,59559.27,2.9423282,7.981417,-2.4735303,"1,5",--,--,--,--,--,--,--,--,--,--
278.54366208675265,63.979596966488785,--,--,HEROES,--,158913789961,--,--,--,--,--,24.422678,0.47287714,24.341719,0.3389037,--,--,--,23.07018,0.34841648,23.06124,0.26967525,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19883.473,59559.066,3.3169792,1.5757403,-0.9627106,"1,5",--,--,--,--,--,--,--,--,--,--
278.557056527257,63.98110038446475,--,--,HEROES,--,158913789966,--,--,--,--,--,23.784037,0.2651253,23.840254,0.2155417,--,--,--,23.326906,0.4468022,23.219593,0.31476638,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19763.244,59610.066,3.554889,4.4261637,-2.4642541,"1,5",--,--,--,--,--,--,--,--,--,--
278.5660856348791,63.98054121651192,--,--,HEROES,--,158913789967,--,--,--,--,--,--,--,--,--,--,--,--,21.020847,0.05430802,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19677.0,59611.0,--,--,--,"1,5",--,--,--,--,--,--,--,--,--,--
278.5422704450695,63.982163849010426,--,--,HEROES,--,158913789968,--,--,--,--,--,23.621317,0.22423156,--,--,--,--,--,23.831797,0.47693807,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19904.959,59611.71,--,--,--,"1,5",--,--,--,--,--,--,--,--,--,--
278.562633433481,63.982529692938314,--,--,HEROES,--,158913789971,--,--,--,--,--,23.6037,0.22494477,23.531757,0.20944446,--,--,--,23.072655,0.35450727,23.254608,0.41856563,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19715.848,59648.402,1.097193,3.530041,1.078596,"1,5",--,--,--,--,--,--,--,--,--,--
278.5502414378729,63.98346593960416,--,--,HEROES,--,158913789972,--,--,--,--,--,23.061796,0.13471395,23.24774,0.22281943,--,--,--,23.019363,0.33279997,22.713615,0.35374734,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,19834.762,59650.727,3.682224,6.6021667,1.3244936,"1,5",--,--,--,--,--,--,--,--,--,--


In [22]:
filteredSubaru = SubaruTable

# Ensure ls_id is masked for invalid values
DESITable['ls_id'] = np.ma.masked_invalid(filteredSubaru['ls_id'])

# Create a mask for valid (non-masked) values
valid_mask = ~DESITable['ls_id'].mask  # True for valid, False for NaN/masked

# Filter table to keep only valid rows
DESITable_filtered = DESITable[valid_mask]

# Print the number of remaining rows
print(f"Filtered table contains {len(DESITable_filtered)} valid rows.")

Filtered table contains 7633408 valid rows.


In [23]:
import numpy as np

# Step 1: Convert int64 to string
DESITable_filtered['ls_id'] = DESITable_filtered['ls_id'].astype(str)

# Step 2: Convert string to bytes (bytes64)
DESITable_filtered['ls_id'] = np.char.encode(DESITable_filtered['ls_id'], encoding='utf-8')

# Step 3: Verify the data type after conversion
print(DESITable_filtered['ls_id'].dtype)  # Should now be bytes64


|S16


In [24]:
# Convert to fixed-length bytes of 64 characters
DESITable_filtered['ls_id'] = DESITable_filtered['ls_id'].astype('|S64')

# Verify the dtype after conversion
print(DESITable_filtered['ls_id'].dtype)  # Should now be '|S64'


|S64


In [25]:
DESITable_filtered

ra_heroes,dec_heroes,ra_ls,dec_ls,from,ls_id,hsc_id,type,i_ap2,i_ap2err,i_kron,i_kronerr,z_ap2,z_ap2err,z_kron,z_kronerr,flux_z,mag_z,snr_z,y_ap2,y_ap2err,y_kron,y_kronerr,n816_ap2,n816_ap2err,n816_kron,n816_kronerr,n921_ap2,n921_ap2err,n921_kron,n921_kronerr,mag_w1,flux_w1,snr_w1,mag_w2,flux_w2,snr_w2,mag_w3,flux_w3,snr_w3,mag_w4,flux_w4,snr_w4,x,y,xx,yy,xy,patch,wise_coadd_id,brickid,brickname,sersic,shape_r,shape_e1,shape_e2,ellipticity,theta,sc_id
float64,float64,float64,float64,bytes6,bytes64,int64,bytes3,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,bytes5,bytes8,int64,bytes8,float64,float64,float64,float64,float64,float64,int64
--,--,280.6799102847307,62.20643145149165,DESI,9907740170586529,--,REX,--,--,--,--,--,--,--,--,0.461153,23.340387,3.514399,--,--,--,--,--,--,--,--,--,--,--,--,22.574854,0.93338,2.755878,21.869234,1.787749,3.015943,19.453398,16.54402,1.081616,--,-148.42885,-1.050232,--,--,--,--,--,--,2810p620,623973,2806p622,1.0,0.237419,0.0,0.0,0.0,0.0,--
--,--,280.1786871652837,61.98809110258671,DESI,9907740126282919,--,REX,--,--,--,--,--,--,--,--,1.085316,22.41111,6.325386,--,--,--,--,--,--,--,--,--,--,--,--,21.484646,2.547663,5.57374,24.416807,0.171111,0.241635,--,-109.156975,-7.570274,13.279389,4878.0273,36.75331,--,--,--,--,--,--,2810p620,623297,2803p620,1.0,0.758481,0.0,0.0,0.0,0.0,--
--,--,280.1792678141988,61.98853350454744,DESI,9907740126282922,--,DEV,--,--,--,--,--,--,--,--,9.51695,20.053755,80.37998,--,--,--,--,--,--,--,--,--,--,--,--,20.125666,8.907044,20.245705,21.563608,2.368953,3.443895,--,-56.93692,-4.019351,13.829157,2939.9326,22.289349,--,--,--,--,--,--,2810p620,623297,2803p620,4.0,0.194002,0.060687,-0.031732,0.06848234657924625,-0.24089169059169896,--
--,--,280.18314613921484,61.99162776035924,DESI,9907740126282954,--,REX,--,--,--,--,--,--,--,--,0.739205,22.828089,5.463509,--,--,--,--,--,--,--,--,--,--,--,--,21.630342,2.227731,6.605207,21.524023,2.456919,4.188169,--,-13.951918,-0.974591,--,-132.43843,-0.993295,--,--,--,--,--,--,2810p620,623297,2803p620,1.0,0.48295,0.0,0.0,0.0,0.0,--
--,--,280.18093982155426,61.99325660421417,DESI,9907740126282938,--,PSF,--,--,--,--,--,--,--,--,2.284047,21.603237,23.173101,--,--,--,--,--,--,--,--,--,--,--,--,22.199097,1.319353,3.931087,26.239449,0.031932,0.055395,--,-15.169825,-1.070138,16.810667,188.68323,1.415503,--,--,--,--,--,--,2810p620,623297,2803p620,0.0,0.0,0.0,0.0,0.0,0.0,--
--,--,280.1654088729099,61.97495634053798,DESI,9907740126282798,--,REX,--,--,--,--,--,--,--,--,1.217714,22.286137,9.741726,--,--,--,--,--,--,--,--,--,--,--,--,20.744257,5.038453,14.554774,21.10697,3.60755,6.110802,--,-22.917543,-1.625457,16.661459,216.47917,1.653989,--,--,--,--,--,--,2810p620,623297,2803p620,1.0,0.22282,0.0,0.0,0.0,0.0,--
--,--,280.1618556487125,61.97765869258246,DESI,9907740126282764,--,PSF,--,--,--,--,--,--,--,--,4.953025,20.762823,42.489613,--,--,--,--,--,--,--,--,--,--,--,--,21.52408,2.456791,7.325959,21.571768,2.351218,4.033747,--,-41.087425,-2.915183,--,-247.66222,-1.896841,--,--,--,--,--,--,2810p620,623297,2803p620,0.0,0.0,0.0,0.0,0.0,0.0,--
--,--,280.1567723570446,61.97696932132687,DESI,9907740126282717,--,PSF,--,--,--,--,--,--,--,--,1.32239,22.196602,11.563542,--,--,--,--,--,--,--,--,--,--,--,--,20.37345,7.089571,20.082256,20.571146,5.909375,9.812005,20.280758,7.721407,0.554505,--,-144.74225,-1.115109,--,--,--,--,--,--,2810p620,623297,2803p620,0.0,0.0,0.0,0.0,0.0,0.0,--
--,--,280.1538037382507,61.97564844939933,DESI,9907740126282691,--,PSF,--,--,--,--,--,--,--,--,4.192486,20.94382,35.989693,--,--,--,--,--,--,--,--,--,--,--,--,20.647339,5.508897,15.802302,21.378927,2.808205,4.764745,19.596403,14.502371,1.040694,16.453838,262.0988,2.016642,--,--,--,--,--,--,2810p620,623297,2803p620,0.0,0.0,

In [42]:
from astropy.table import Table, join

# Step 1: Make a copy of mergedTablev1
mergedTablev1_copy = mergedTablev1.copy()

# Step 2: Add a placeholder column to track original rows
mergedTablev1_copy['placeholder'] = 1

# Step 3: Define columns to append
columns_to_append = ["type", "sersic", "shape_r", "shape_e1", "shape_e2", "ellipticity", "theta"]

# Step 4: Perform left join on "ls_id"
merged_final = join(mergedTablev1_copy, DESITable_filtered['ls_id', *columns_to_append], 
                    keys="ls_id", join_type="left")

# Step 5: Remove extra rows (where placeholder != 1)
merged_final = merged_final[merged_final['placeholder'] == 1]

# Step 6: Drop the placeholder column
merged_final.remove_column('placeholder')



In [44]:
merged_final.sort('id')

In [45]:
merged_final2 = merged_final
merged_final2

band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,id,ra,dec,z_best,z_med,z_err_std,snr_per_filter,ls_bit,unwise_bit,h20_bit,sv3_bit,heroes_bit,ls_id,sls_bit,unwise_id,h20_id,sv3_id,heroes_id,sls_id,type,sersic,shape_r,shape_e1,shape_e2,ellipticity,theta
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,bytes64,int64,bytes64,bytes64,bytes64,bytes64,float64,bytes3,float64,float64,float64,float64,float64,float64
2.681520900686674e-05,3.048576225348575e-05,4.262261112106731e-05,5.407932189180571e-05,5.9616764561542935e-05,5.070858833667531e-05,4.086165978041212e-05,3.7154243476732574e-05,3.5661351391208783e-05,3.4767379294930216e-05,2,280.17962646484375,61.98339080810547,0.30530010018849013,0.3070000112056732,0.02167546741850781,8.988694190979004,1,1,0,0,0,9907740126282928.0,0,2810p620o0018688,nan,nan,nan,--,--,--,--,--,--,--,--
5.411545241695075e-06,8.762154299498739e-06,1.5490201241545152e-05,1.7676723609592366e-05,1.8253706283506807e-05,2.3256594545698388e-05,2.755236119193311e-05,2.9913880375036387e-05,3.313240689251029e-05,3.464740964024784e-05,3,280.1791687011719,61.98342514038086,1.7943031981142665,1.7929999828338623,0.7636348248884954,5.643349647521973,1,1,0,0,0,9907740126282920.0,0,2810p620o0018688,nan,nan,nan,--,--,--,--,--,--,--,--
1.9800170292624555e-05,3.565085085493113e-05,5.098289028396336e-05,6.790073840029793e-05,7.756522588685293e-05,9.12520180844763e-05,7.842414361315851e-05,5.336935850298715e-05,4.588528572930254e-05,4.4097709944449394e-05,5,280.190673828125,61.984432220458984,0.679355981752951,0.6740000247955322,0.07312436718989827,7.261623382568359,1,1,0,0,0,9907740126283024.0,0,2810p620o0018707,nan,nan,nan,--,--,--,--,--,--,--,--
7.510220454562847e-06,1.150310060009293e-05,2.1039755066609233e-05,4.460379914448898e-05,5.878832381575617e-05,7.460214649641206e-05,9.412971835377377e-05,9.767767350602094e-05,0.00010009535898087219,0.00010548488206710966,9,280.18048095703125,61.98760986328125,2.6019985018422793,2.5139999389648438,0.7484448868558956,5.81927490234375,1,1,0,0,0,9907740126282936.0,0,2810p620o0056304,nan,nan,nan,--,--,--,--,--,--,--,--
8.562705235041022e-05,8.011751240164219e-05,0.00010005936984396459,0.00011113417898552091,0.00012175907358478118,0.00011098486821829784,9.017115178636255e-05,8.274301618677838e-05,7.977027400907374e-05,7.802582936474643e-05,10,280.1800842285156,61.987831115722656,0.4170768654944781,0.4189999997615814,0.0067790824098721844,23.856380462646484,1,1,0,0,0,9907740126282932.0,0,2810p620o0056304,nan,nan,nan,--,--,--,--,--,--,--,--
2.2033035957456458e-05,3.337165419660239e-05,4.920775023874026e-05,6.243579787890124e-05,7.289114820847241e-05,6.195681891131399e-05,4.653173590978113e-05,4.596466602770599e-05,3.836609154896328e-05,3.441837330986877e-05,11,280.15484619140625,61.96268081665039,0.2699319741423631,0.2630000114440918,0.17263437585354416,6.052838325500488,1,1,0,0,0,9907740126282702.0,0,2810p620o0056064,nan,nan,nan,--,--,--,--,--,--,--,--
3.4913219181948615e-05,6.164957706929101e-05,9.045183728050391e-05,0.00011400111717445002,0.00014613313161225392,0.00011834347881993089,7.564400002685087e-05,7.144664219501903e-05,6.532508472259995e-05,5.729920453428324e-05,12,280.14947509765625,61.964317321777344,0.24544961424590403,0.23800000548362732,0.07912544380225438,9.083745002746582,1,1,0,0,0,9907740126282640.0,0,2810p620o0070791,nan,nan,nan,--,--,--,--,--,--,--,--
9.264645814469688e-05,9.189179821675676e-05,8.875463409124713e-05,8.813442056562035e-05,9.546471824034616e-05,0.00012097472101063973,8.132825160616305e-05,0.00020132196527433206,8.577983740573803e-05,8.384796833406042e-05,21,280.16021728515625,61.98466873168945,4.990999917392784,0.0,5.923168878389097e-05,16.59946632385254,1,1,0,0,0,9907740126282748.0,0,2810p620o0018708,nan,nan,nan,--,--,--,--,--,--,--,--
1.823168666343588e-05,3.93529311476616e-05,7.162970341369655e-05,0.00010566256342355037,0.00012

In [46]:
print(merged_final2.colnames)


['band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10', 'id', 'ra', 'dec', 'z_best', 'z_med', 'z_err_std', 'snr_per_filter', 'ls_bit', 'unwise_bit', 'h20_bit', 'sv3_bit', 'heroes_bit', 'ls_id', 'sls_bit', 'unwise_id', 'h20_id', 'sv3_id', 'heroes_id', 'sls_id', 'type', 'sersic', 'shape_r', 'shape_e1', 'shape_e2', 'ellipticity', 'theta']


In [47]:
# Define the desired column order
desired_order = [
    'id', 'ra', 'dec', 'snr_per_filter', 'z_best', 'z_med', 'z_err_std',
    'band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
    'type', 'sersic', 'shape_r', 'shape_e1', 'shape_e2', 'ellipticity', 'theta',
    'ls_bit', 'unwise_bit', 'h20_bit', 'sv3_bit', 'heroes_bit', 'ls_id',
    'sls_bit', 'unwise_id', 'h20_id', 'sv3_id', 'heroes_id', 'sls_id'
]

# Reorder columns (ensuring all exist in the table)
merged_final2 = merged_final2[desired_order]

# Confirm new order
print("✅ Table columns successfully reordered.")
print("New column order:", merged_final2.colnames)


✅ Table columns successfully reordered.
New column order: ['id', 'ra', 'dec', 'snr_per_filter', 'z_best', 'z_med', 'z_err_std', 'band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10', 'type', 'sersic', 'shape_r', 'shape_e1', 'shape_e2', 'ellipticity', 'theta', 'ls_bit', 'unwise_bit', 'h20_bit', 'sv3_bit', 'heroes_bit', 'ls_id', 'sls_bit', 'unwise_id', 'h20_id', 'sv3_id', 'heroes_id', 'sls_id']


In [48]:
merged_final2

id,ra,dec,snr_per_filter,z_best,z_med,z_err_std,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,type,sersic,shape_r,shape_e1,shape_e2,ellipticity,theta,ls_bit,unwise_bit,h20_bit,sv3_bit,heroes_bit,ls_id,sls_bit,unwise_id,h20_id,sv3_id,heroes_id,sls_id
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes3,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,bytes64,int64,bytes64,bytes64,bytes64,bytes64,float64
2,280.17962646484375,61.98339080810547,8.988694190979004,0.30530010018849013,0.3070000112056732,0.02167546741850781,2.681520900686674e-05,3.048576225348575e-05,4.262261112106731e-05,5.407932189180571e-05,5.9616764561542935e-05,5.070858833667531e-05,4.086165978041212e-05,3.7154243476732574e-05,3.5661351391208783e-05,3.4767379294930216e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282928.0,0,2810p620o0018688,nan,nan,nan,--
3,280.1791687011719,61.98342514038086,5.643349647521973,1.7943031981142665,1.7929999828338623,0.7636348248884954,5.411545241695075e-06,8.762154299498739e-06,1.5490201241545152e-05,1.7676723609592366e-05,1.8253706283506807e-05,2.3256594545698388e-05,2.755236119193311e-05,2.9913880375036387e-05,3.313240689251029e-05,3.464740964024784e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282920.0,0,2810p620o0018688,nan,nan,nan,--
5,280.190673828125,61.984432220458984,7.261623382568359,0.679355981752951,0.6740000247955322,0.07312436718989827,1.9800170292624555e-05,3.565085085493113e-05,5.098289028396336e-05,6.790073840029793e-05,7.756522588685293e-05,9.12520180844763e-05,7.842414361315851e-05,5.336935850298715e-05,4.588528572930254e-05,4.4097709944449394e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126283024.0,0,2810p620o0018707,nan,nan,nan,--
9,280.18048095703125,61.98760986328125,5.81927490234375,2.6019985018422793,2.5139999389648438,0.7484448868558956,7.510220454562847e-06,1.150310060009293e-05,2.1039755066609233e-05,4.460379914448898e-05,5.878832381575617e-05,7.460214649641206e-05,9.412971835377377e-05,9.767767350602094e-05,0.00010009535898087219,0.00010548488206710966,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282936.0,0,2810p620o0056304,nan,nan,nan,--
10,280.1800842285156,61.987831115722656,23.856380462646484,0.4170768654944781,0.4189999997615814,0.0067790824098721844,8.562705235041022e-05,8.011751240164219e-05,0.00010005936984396459,0.00011113417898552091,0.00012175907358478118,0.00011098486821829784,9.017115178636255e-05,8.274301618677838e-05,7.977027400907374e-05,7.802582936474643e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282932.0,0,2810p620o0056304,nan,nan,nan,--
11,280.15484619140625,61.96268081665039,6.052838325500488,0.2699319741423631,0.2630000114440918,0.17263437585354416,2.2033035957456458e-05,3.337165419660239e-05,4.920775023874026e-05,6.243579787890124e-05,7.289114820847241e-05,6.195681891131399e-05,4.653173590978113e-05,4.596466602770599e-05,3.836609154896328e-05,3.441837330986877e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282702.0,0,2810p620o0056064,nan,nan,nan,--
12,280.14947509765625,61.964317321777344,9.083745002746582,0.24544961424590403,0.23800000548362732,0.07912544380225438,3.4913219181948615e-05,6.164957706929101e-05,9.045183728050391e-05,0.00011400111717445002,0.00014613313161225392,0.00011834347881993089,7.564400002685087e-05,7.144664219501903e-05,6.532508472259995e-05,5.729920453428324e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282640.0,0,2810p620o0070791,nan,nan,nan,--
21,280.16021728515625,61.98466873168945,16.59946632385254,4.990999917392784,0.0,5.923168878389097e-05,9.264645814469688e-05,9.189179821675676e-05,8.875463409124713e-05,8.813442056562035e-05,9.546471824034616e-05,0.00012097472101063973,8.132825160616305e-05,0.00020132196527433206,8.577983740573803e-05,8.384796833406042e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282748.0,0,2810p620o0018708,nan,nan,nan,--
29,280.1792907714844,61.98844909667969,9.050853729248047,0.1218945147299026,0.09799999743700027

In [50]:
from astropy.io import fits

# Define the save path
save_path = "/mnt/md127/SPHEREx/largeOutputs/spherex_flux_10binned.fits"

# Save the table as a FITS file
merged_final2.write(save_path, format="fits", overwrite=True)

print(f"✅ Table saved successfully at: {save_path}")


✅ Table saved successfully at: /mnt/md127/SPHEREx/largeOutputs/spherex_flux_10binned.fits


In [ ]:
testCat = Table.read(save_path)
testCat

id,ra,dec,snr_per_filter,z_best,z_med,z_err_std,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,type,sersic,shape_r,shape_e1,shape_e2,ellipticity,theta,ls_bit,unwise_bit,h20_bit,sv3_bit,heroes_bit,ls_id,sls_bit,unwise_id,h20_id,sv3_id,heroes_id,sls_id
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes3,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,bytes64,int64,bytes64,bytes64,bytes64,bytes64,float64
2,280.17962646484375,61.98339080810547,8.988694190979004,0.30530010018849013,0.3070000112056732,0.02167546741850781,2.681520900686674e-05,3.048576225348575e-05,4.262261112106731e-05,5.407932189180571e-05,5.9616764561542935e-05,5.070858833667531e-05,4.086165978041212e-05,3.7154243476732574e-05,3.5661351391208783e-05,3.4767379294930216e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282928.0,0,2810p620o0018688,nan,nan,nan,--
3,280.1791687011719,61.98342514038086,5.643349647521973,1.7943031981142665,1.7929999828338623,0.7636348248884954,5.411545241695075e-06,8.762154299498739e-06,1.5490201241545152e-05,1.7676723609592366e-05,1.8253706283506807e-05,2.3256594545698388e-05,2.755236119193311e-05,2.9913880375036387e-05,3.313240689251029e-05,3.464740964024784e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282920.0,0,2810p620o0018688,nan,nan,nan,--
5,280.190673828125,61.984432220458984,7.261623382568359,0.679355981752951,0.6740000247955322,0.07312436718989827,1.9800170292624555e-05,3.565085085493113e-05,5.098289028396336e-05,6.790073840029793e-05,7.756522588685293e-05,9.12520180844763e-05,7.842414361315851e-05,5.336935850298715e-05,4.588528572930254e-05,4.4097709944449394e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126283024.0,0,2810p620o0018707,nan,nan,nan,--
9,280.18048095703125,61.98760986328125,5.81927490234375,2.6019985018422793,2.5139999389648438,0.7484448868558956,7.510220454562847e-06,1.150310060009293e-05,2.1039755066609233e-05,4.460379914448898e-05,5.878832381575617e-05,7.460214649641206e-05,9.412971835377377e-05,9.767767350602094e-05,0.00010009535898087219,0.00010548488206710966,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282936.0,0,2810p620o0056304,nan,nan,nan,--
10,280.1800842285156,61.987831115722656,23.856380462646484,0.4170768654944781,0.4189999997615814,0.0067790824098721844,8.562705235041022e-05,8.011751240164219e-05,0.00010005936984396459,0.00011113417898552091,0.00012175907358478118,0.00011098486821829784,9.017115178636255e-05,8.274301618677838e-05,7.977027400907374e-05,7.802582936474643e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282932.0,0,2810p620o0056304,nan,nan,nan,--
11,280.15484619140625,61.96268081665039,6.052838325500488,0.2699319741423631,0.2630000114440918,0.17263437585354416,2.2033035957456458e-05,3.337165419660239e-05,4.920775023874026e-05,6.243579787890124e-05,7.289114820847241e-05,6.195681891131399e-05,4.653173590978113e-05,4.596466602770599e-05,3.836609154896328e-05,3.441837330986877e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282702.0,0,2810p620o0056064,nan,nan,nan,--
12,280.14947509765625,61.964317321777344,9.083745002746582,0.24544961424590403,0.23800000548362732,0.07912544380225438,3.4913219181948615e-05,6.164957706929101e-05,9.045183728050391e-05,0.00011400111717445002,0.00014613313161225392,0.00011834347881993089,7.564400002685087e-05,7.144664219501903e-05,6.532508472259995e-05,5.729920453428324e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282640.0,0,2810p620o0070791,nan,nan,nan,--
21,280.16021728515625,61.98466873168945,16.59946632385254,4.990999917392784,0.0,5.923168878389097e-05,9.264645814469688e-05,9.189179821675676e-05,8.875463409124713e-05,8.813442056562035e-05,9.546471824034616e-05,0.00012097472101063973,8.132825160616305e-05,0.00020132196527433206,8.577983740573803e-05,8.384796833406042e-05,--,--,--,--,--,--,--,1,1,0,0,0,9907740126282748.0,0,2810p620o0018708,nan,nan,nan,--
29,280.1792907714844,61.98844909667969,9.050853729248047,0.1218945147299026,0.09799999743700027

: 